In [1]:
import pandas as pd
import numpy as np

In [2]:
SCREENINGFILE = "ED_Screening_Main_March2023_April+7,+2023_15.40.csv"
df_screen = pd.read_csv("..\data\{}".format(SCREENINGFILE))

QUALFILE = "ED_Main_Spring_2023_April+7,+2023_15.48.csv"
df_qual = pd.read_csv("..\data\{}".format(QUALFILE))

KEYFILE = "key.csv"
df_key = pd.read_csv("..\data\{}".format(KEYFILE))

In [3]:
df_qual[df_qual.ID == "6032"] # R_2AFqSO4EGg2I8eJ should be 9377

,StartDate,EndDate,Status,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,DistributionChannel,UserLanguage,...,Q15_Page Submit,Q15_Click Count,t_drink_First Click,t_drink_Last Click,t_drink_Page Submit,t_drink_Click Count,Q1_1,Q1_2,COGED_s5,ID.1
36,2023-03-08 09:25:30,2023-03-08 09:40:13,0,100,882,1,2023-03-08 09:40:13,R_2AFqSO4EGg2I8eJ,anonymous,EN,...,20.108,0,0,0,120.01,0,5,2,4,NaN
48,2023-03-08 13:48:50,2023-03-08 14:14:20,0,100,1530,1,2023-03-08 14:14:21,R_Qj4BdPjLUzjY9Rn,anonymous,EN,...,20.011,0,0,0,120.011,0,5,1,3,NaN
106,2023-03-15 15:16:17,2023-03-15 15:32:50,0,100,992,1,2023-03-15 15:32:50,R_2wnsuSL25OTE4ma,anonymous,EN,...,20.014,0,0,0,120.125,0,5,0,3,NaN
190,2023-03-23 12:17:15,2023-03-23 13:55:47,0,100,5911,1,2023-03-23 13:55:48,R_3Ed14Er8d8XmXVR,anonymous,EN,...,20.007,0,12.464,12.464,120.012,1,6,7,3,NaN


In [33]:
# need to work out which of these is correct T1 session
df = df_qual
df[df.ID == "5026"]

,StartDate,EndDate,Status,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,DistributionChannel,UserLanguage,...,Q15_Page Submit,Q15_Click Count,t_drink_First Click,t_drink_Last Click,t_drink_Page Submit,t_drink_Click Count,Q1_1,Q1_2,COGED_s5,ID.1
58,2023-03-09 09:02:31,2023-03-09 09:44:00,0,100,2489,1,2023-03-09 09:44:01,R_cIwJxoHng65K0Zr,anonymous,EN,...,20.116,0,0,0,120.114,0,8,1,3,NaN
60,2023-03-09 08:59:10,2023-03-09 09:49:13,0,100,3003,1,2023-03-09 09:49:14,R_1qVeJjadTwlXHy7,anonymous,EN,...,20.008,0,0,0,120.012,0,6,5,5,NaN
131,2023-03-16 15:25:45,2023-03-16 15:36:27,0,100,641,1,2023-03-16 15:36:27,R_25tQId1LawccDP6,anonymous,EN,...,20.113,0,0,0,120.121,0,6,9,4,NaN
170,2023-03-22 15:23:08,2023-03-22 15:34:07,0,100,658,1,2023-03-22 15:34:09,R_330Sovu3EVJKSjL,anonymous,EN,...,20.014,0,0,0,120.015,0,6,7,5,NaN


In [34]:
def tweak_key(df):
    return(df
           .drop(df.index[df.ID == 2018])
           .assign(ID=lambda df_: df_.ID.astype(str))
           .melt(id_vars="ID")
           .rename(columns={"variable": "session",
                            "value": "condition"})
           .assign(session=lambda df_: df_.session.astype("category"),
                   condition=lambda df_: df_.condition.astype("category"))
          )

def tweak_panas(df, df_key):
    IDs = df_key.ID.unique()
    panas_cols = ["ID", "session"] + [col for col in df.columns if "PANAS" in col]
    
    return(df
           .assign(ID=lambda df_: np.where(df_qual.ResponseId == "R_2AFqSO4EGg2I8eJ", "9377", df_qual.ID))
           #.assign(ID=lambda df_: np.where...) # line to fix ID 5026
           [panas_cols]
           .query("ID in @IDs")
           .assign(session=lambda df_: "T" + df_.session)
           .astype({col:"uint8" for col in df_qual.columns if "PANAS" in col})
           .astype({"ID":"category"})
           .merge(df_key)
          )

key = tweak_key(df_key)
panas = tweak_panas(df_qual, key)

In [27]:
p2 = (panas.groupby("ID").size() == 2)
p2[p2 == True]
# panas[panas.ID == "5026"]

ID
4098    True
9102    True
9170    True
dtype: bool

In [32]:
df = df_qual
s = df.session
s[df.ID == "9170"]

86     1
150    2
Name: session, dtype: object

In [5]:
panas.to_feather("..\\data\\panas.fea")

In [6]:
p2 = pd.read_feather("..\\data\\panas.fea")

In [11]:
p2.describe()

,PANAS_1,PANAS_2,PANAS_3,PANAS_4,PANAS_5,PANAS_6,PANAS_7,PANAS_8,PANAS_9,PANAS_10,PANAS_11,PANAS_12,PANAS_13,PANAS_14,PANAS_15
count,197.000000,197.000000,197.000000,197.000000,197.000000,197.000000,197.000000,197.000000,197.000000,197.000000,197.000000,197.000000,197.000000,197.000000,197.000000
mean,2.949239,2.629442,1.309645,2.913706,2.020305,3.380711,3.055838,1.472081,2.741117,1.649746,2.426396,1.248731,1.862944,2.583756,2.944162
std,0.861573,1.078428,0.631291,0.849769,0.936556,0.893419,0.852325,0.746066,0.941658,0.865532,0.904024,0.557015,0.945719,1.147019,0.921362
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,2.000000,2.000000,1.000000,2.000000,1.000000,3.000000,3.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000,2.000000,2.000000
50%,3.000000,2.000000,1.000000,3.000000,2.000000,3.000000,3.000000,1.000000,3.000000,1.000000,2.000000,1.000000,2.000000,2.000000,3.000000
75%,4.000000,3.000000,1.000000,3.000000,3.000000,4.000000,4.000000,2.000000,3.000000,2.000000,3.000000,1.000000,2.000000,4.000000,4.000000
max,5.000000,5.000000,4.000000,5.000000,4.000000,5.000000,5.000000,4.000000,5.000000,4.000000,5.000000,4.000000,5.000000,5.000000,5.000000


{'PANAS_1': 'category',
 'PANAS_2': 'category',
 'PANAS_3': 'category',
 'PANAS_4': 'category',
 'PANAS_5': 'category',
 'PANAS_6': 'category',
 'PANAS_7': 'category',
 'PANAS_8': 'category',
 'PANAS_9': 'category',
 'PANAS_10': 'category',
 'PANAS_11': 'category',
 'PANAS_12': 'category',
 'PANAS_13': 'category',
 'PANAS_14': 'category',
 'PANAS_15': 'category'}